# 103 LangGraph: Your First Graph - Foundation

**Workshop**: LangGraph 101 - Foundations
**Duration**: ~20 minutes
**Difficulty**: Beginner

## Prerequisites

- **Knowledge**: Completed Notebooks 101 (Type Annotations) and 102 (Core Concepts)
- **Setup**: None required - no LLM APIs needed for this workshop

## Learning Objectives

By completing this notebook, you will:
- Define agent state structure using TypedDict
- Create simple node functions that process and update state
- Build your first basic LangGraph structure
- Understand how to compile and invoke graphs
- See how data flows through a single node in LangGraph
- Apply these concepts to validate SCM address objects

## Table of Contents

1. [Introduction](#1-introduction)
2. [Defining State Structure](#2-defining-state-structure)
3. [Creating Node Functions](#3-creating-node-functions)
4. [Building the Graph](#4-building-the-graph)
5. [Compiling and Executing](#5-compiling-and-executing)
6. [Exercise: SCM Address Object Validation](#51-exercise-scm-address-object-validation)
7. [Summary](#6-summary)
8. [What's Next](#7-whats-next)


## 1. Introduction

Welcome to your first hands-on LangGraph coding workshop! This is where we transition from theory to practice.

### A Quick Confession

I have a confession to make: **we're not building AI agents yet**. Here's why:

We haven't actually written any LangGraph code yet, and jumping straight into combining LLMs, APIs, tools, and complex workflows would be messy and confusing. This workshop is designed to be beginner-friendly, detailed, and comprehensive - we're going to take it step by step.

Don't worry - we'll be coding AI agents soon! Right now, we're going to:
1. Build a simple graph
2. Understand LangGraph syntax and patterns
3. Get confident with how graphs work
4. **Then** we'll build more complex workflows (in later notebooks)

### The "Hello World" Graph

What we're building today is what I call the **Hello World Graph** - the most basic form of graph you can build in LangGraph.

**Objectives for this workshop:**
- Understand and define agent state structure
- Create simple node functions that process and update state
- Build your first basic LangGraph structure
- Learn how to compile, invoke, and process graphs
- **Main goal**: Understand how data flows through a single node in LangGraph

### What We're Building

Here's a preview of the graph structure:

```
START → validate_address_object → END
```

This is the simplest possible graph:
- A **START** point (entry)
- A **single node** in the middle (validate)
- An **END** point (exit)

Think of it like a **basic SCM address object validation workflow**:
- You start the validation
- Process address object information
- Complete the validation

Simple, but it will teach you the fundamental mechanics of LangGraph!

Let's get started!


### 1.1 Imports

Let's start by importing the three main components we need:

In [ ]:
# Core imports
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

# Visualization
from IPython.display import Image, display

print("✅ Imports successful!")
print("\nWhat we imported:")
print("  - TypedDict: For defining our state structure")
print("  - StateGraph: Framework to design and manage task flow")
print("  - START/END: Entry and exit points for our graph")
print("  - IPython display: To visualize our graph")

---

## 2. Defining State Structure

The first thing we need to do is create the **state** of our workflow. Let's call it `FirewallCheckState`.

**Quick Refresher**: Think of state as a shared data structure that keeps track of all your information as the workflow runs. It's like tracking device information during a basic firewall health check.

We build state using a class that inherits from `TypedDict`. Let's keep this very fundamental and basic - we'll use just two fields: `hostname` and `status`.

In [ ]:
class AddressObjectState(TypedDict):
    """State for tracking SCM address object information."""
    name: str      # Address object name
    status: str    # Validation status

print("✅ AddressObjectState defined!")
print("\nState structure:")
print("  - name: str      (address object name)")
print("  - status: str    (validation status)")
print("\n💡 This is just normal Python - a class inheriting from TypedDict")

---

## 3. Creating Node Functions

Now we're going to code our very first **node** - another fundamental element in LangGraph.

**How do we define a node?** It's quite simple - it's just a normal standard Python function!

Let's create a simple device status check node. Here's what we need to know:
- **Input type**: Must be the state (because state tracks all information)
- **Output type**: Must be a dict (partial state update)

### Why Docstrings Matter (Especially for AI Agents!)

Docstrings are **very important** in LangGraph. Here's why:

1. **For Human Developers**: Docstrings explain what your function does, making code maintainable

2. **For AI Agents** (coming in later notebooks): When you build AI agents with LLMs, they read docstrings to understand:
   - What each tool/function does
   - What parameters it accepts
   - What it returns
   - When to use it

**Example**: When we build an AI agent in Notebook 106, if you have:
```python
def validate_address_object(state):
    \"\"\"Validate SCM address object configuration.
    
    Checks that address object name follows conventions and is ready for creation.
    \"\"\"
```

The AI agent reads this docstring and understands: "This function validates address objects - I should use it when the user asks to validate a configuration before creating it!"

Without the docstring, the AI agent doesn't know what your function does or when to use it.

### Creating Your First Node

To create a docstring: use three quotation marks `\"\"\"`

Let's build our validation node:

In [ ]:
def validate_address_object(state: AddressObjectState) -> dict:
    """Node: Validate SCM address object configuration.

    This function represents a single 'node' in our LangGraph workflow.
    It takes the current state as input and returns a partial state update.

    Args:
        state: Current state containing address object information

    Returns:
        dict: Partial state update with validation status
    """
    # Read the address object name from state
    address_name = state["name"]

    # Simulate validation logic (in production, this would validate via SCM API)
    validation_result = f"Address object '{address_name}' validated successfully in SCM"

    # Return ONLY the field we're updating (partial state update)
    return {"status": validation_result}

print("✅ validate_address_object function defined!")
print("\n💡 Key Pattern: Functions take state as input, return dict with updates")

### 🏭 Production Pattern: What Real SCM Validation Looks Like

**Important**: The function above is simplified for learning. In production, you'd validate against the actual SCM API. Here's what that would look like:

```python
# NOT EXECUTED - Just showing the production pattern
from scm.client import ScmClient
from scm.exceptions import ObjectNotPresentError, InvalidObjectError

def validate_address_object_production(state: AddressObjectState) -> dict:
    """Node: Validate SCM address object (production version).
    
    Checks if an address object name is available for creation
    by attempting to fetch it from SCM. If not found, it's available.
    """
    try:
        # Initialize SCM client (in production, do this once and reuse)
        client = ScmClient(
            client_id="your_client_id",
            client_secret="your_client_secret", 
            tsg_id="your_tsg_id"
        )
        
        address_name = state["name"]
        
        # Try to fetch the address object
        # If it exists, we get the object back
        # If it doesn't exist, ObjectNotPresentError is raised
        try:
            existing = client.address.fetch(name=address_name, folder="Texas")
            # If we get here, the address already exists
            status = f"Address object '{address_name}' already exists in SCM (ID: {existing.id})"
            
        except ObjectNotPresentError:
            # This exception means the address doesn't exist - perfect!
            # The name is available for creation
            status = f"Address object '{address_name}' is available for creation"
            
    except InvalidObjectError as e:
        # Configuration data format is wrong
        status = f"Invalid configuration: {e.message}"
    except Exception as e:
        # Catch-all for other errors (network issues, auth failures, etc.)
        status = f"Validation failed: {str(e)}"
    
    return {"status": status}
```

**Key Differences from Learning Version:**

1. **Real API Client**: Initializes `ScmClient` with credentials
2. **Exception Handling**: Uses try-catch for SCM-specific errors
3. **Actual Validation**: Calls `client.address.fetch()` to check if name exists
4. **Error Recovery**: Gracefully handles failures and updates state accordingly

**Why we don't run this now:**
- ❌ Requires API credentials (client_id, client_secret, tsg_id)
- ❌ Requires network connection to SCM
- ❌ More complex for first graph lesson

**When you'll use this:**
- ✅ Notebooks 104-107: We'll add actual SCM operations
- ✅ Production automation: Every real workflow needs this pattern
- ✅ Notebooks 110-111: AI agents will call these as tools

💡 **Remember**: The pattern is the same (state → process → return dict), but production adds API calls and error handling!

### 🛡️ Basic Error Handling Example

Let's add a simple error handling example to demonstrate the pattern. This won't call the SCM API, but shows how to structure try-catch in nodes:



In [ ]:
def validate_with_error_handling(state: AddressObjectState) -> dict:
    """Node: Validate address object with error handling.
    
    Demonstrates basic try-catch pattern for robust node functions.
    This pattern will be essential when working with SCM API calls.
    """
    try:
        # Validate that we have the required fields
        address_name = state["name"]
        
        # Simulate validation checks
        if not address_name:
            raise ValueError("Address name cannot be empty")
        
        if len(address_name) > 63:
            raise ValueError("Address name must be 63 characters or less")
        
        # Validation successful
        status = f"✅ Address object '{address_name}' passed validation checks"
        
    except KeyError as e:
        # Missing required field in state
        status = f"❌ Error: Missing required field {e}"
        
    except ValueError as e:
        # Validation rule failed
        status = f"❌ Validation failed: {str(e)}"
        
    except Exception as e:
        # Catch-all for unexpected errors
        status = f"❌ Unexpected error: {type(e).__name__}: {str(e)}"
    
    # Always return a dict with status, even if there was an error
    return {"status": status}


# Test the error handling with different scenarios
print("Testing error handling with different inputs:")
print("=" * 60)

# Test 1: Valid input
test_cases = [
    {"name": "web_server", "status": "", "description": "Valid name"},
    {"name": "", "status": "", "description": "Empty name (should fail)"},
    {"name": "a" * 64, "status": "", "description": "Name too long (should fail)"},
]

for test_case in test_cases:
    result = validate_with_error_handling(test_case)
    print(f"\nTest: {test_case['description']}")
    print(f"Input: name='{test_case['name']}'")
    print(f"Result: {result['status']}")

print("\n" + "=" * 60)
print("💡 Key Takeaway: Error handling ensures nodes always return valid state")
print("   Even when validation fails, the graph can continue and route appropriately")

---

## 4. Building the Graph

Now let's actually build the graph! Remember, **StateGraph** is a framework that helps us design and manage the flow of tasks as a graph.

### Step 1: Create the Graph

To create a graph, we use `StateGraph` and pass in our state schema:

In [ ]:
# Create the graph with our state schema
graph = StateGraph(AddressObjectState)

print("✅ StateGraph created!")
print("\n💡 The graph now knows about our AddressObjectState schema")

### Step 2: Add a Node

How do we add a node to the graph? We use the built-in function `graph.add_node()`.

It requires **two parameters**:
1. **Name** of your node (can be anything sensible)
2. **Action** it will perform (the function to execute)

In [ ]:
# Add the node to the graph
graph.add_node("validate", validate_address_object)

print("✅ Node added to graph!")
print("\n💡 We've registered our validation function as a node named 'validate'")

### Step 3: Add START and END Points

Remember our diagram? It has a START point, the node, and an END point. We've created the node, but we haven't added START and END yet.

**How do we connect them?**

We use `set_entry_point()` and `set_finish_point()`:
- **set_entry_point**: Connects START to your node
- **set_finish_point**: Connects your node to END

Both methods need the **node name** as a parameter.

In [ ]:
# Connect START → validate
graph.set_entry_point("validate")

# Connect validate → END
graph.set_finish_point("validate")

print("✅ Entry and finish points set!")
print("\n💡 Flow: START → validate → END")

### 📚 Modern LangGraph Pattern: add_edge()

**Quick Note**: We used `set_entry_point()` and `set_finish_point()` above because they're simple and intuitive for beginners. However, there's another pattern you'll see in modern LangGraph code (2025+):

```python
# Alternative pattern using add_edge() - also valid!
graph.add_edge(START, "validate")  # Connect START → validate
graph.add_edge("validate", END)    # Connect validate → END
```

**Both patterns work perfectly!** Here's the difference:

| Pattern | Best For | Why? |
|---------|----------|------|
| `set_entry_point()` / `set_finish_point()` | Simple, single-path graphs | Clearer intent, easier to read for beginners |
| `add_edge()` | Complex graphs with multiple paths | More flexible, explicit edge control |

**Which should you use?**
- For this notebook and simple graphs: Either pattern is fine!
- For Notebooks 104-107 (multi-node, conditional routing): `add_edge()` becomes more important
- For production code: `add_edge()` is the modern standard

**Looking ahead**: In Notebook 106 (Conditional Routing), you'll learn conditional edges where `add_edge()` really shines:
```python
# This flexibility comes with add_edge() pattern
graph.add_conditional_edges(
    "validate",
    route_function,  # Decides which path to take
    {
        "success": "create",
        "error": "retry"
    }
)
```

💡 **Pro Tip**: We're teaching you `set_entry_point()` now for clarity, but you'll graduate to `add_edge()` as graphs get complex. Both are valid LangGraph syntax!

---

## 5. Compiling and Executing

### Step 1: Compile the Graph

One last thing we need to do is **compile** the graph using the built-in `compile()` function:

⚠️ **Word of Caution**: Just because the graph compiles without errors doesn't mean it will successfully run! As we build more complicated graphs, there could be logical errors. So don't get too happy when it compiles - there might still be issues. Trust me, I know!

In [ ]:
# Compile the graph
app = graph.compile()

print("✅ Graph compiled successfully!")
print("\n⚠️  Remember: Successful compilation ≠ guaranteed execution")
print("   There could still be logical errors in complex graphs")

### Step 2: Visualize the Graph

Let's visualize what we built using IPython:

In [ ]:
# Visualize the graph structure
display(Image(app.get_graph().draw_mermaid_png()))

print("\n💡 This diagram shows:")
print("   - __start__: Entry point")
print("   - validate: Our validation node")
print("   - __end__: Exit point")
print("\n   Notice the node name is 'validate' - exactly what we specified!")
print("   The arrows show the flow: START → validate → END")

### Step 3: Run the Graph!

Let's actually run this! To run, we use the built-in method `invoke()`:

In [ ]:
# Run the graph with an initial address object name
result = app.invoke({"name": "web_server_01", "status": ""})

# Access the updated fields from the result
print("Address Name:", result["name"])
print("Status:", result["status"])

print("\n✅ Success! The graph executed and returned the result!")
print("\nHow it works:")
print("  1. We passed in initial state: {'name': 'web_server_01', 'status': ''}")
print("  2. The validate_address_object node processed it")
print("  3. Node read name and generated status message")
print("  4. Final result - name:", result["name"])
print("  5. Final result - status:", result["status"])


### Step 4: Understanding the Flow

Let's break down what just happened step-by-step:

```
┌─────────────────────────────────────────────────────────────┐
│ Initial State                                                │
│ {"name": "web_server_01", "status": ""}                     │
└──────────────────┬──────────────────────────────────────────┘
                   │
                   ▼
           ┌───────────────┐
           │  START point  │
           └───────┬───────┘
                   │
                   ▼
    ┌──────────────────────────────────────┐
    │  validate node                       │
    │  (validate_address_object function)  │
    │                                      │
    │  Processing:                         │
    │  1. Read name: "web_server_01"      │
    │  2. Generate status message         │
    │  3. Return partial update:          │
    │     {"status": "Address object..."}  │
    └──────────────┬───────────────────────┘
                   │
                   ▼
            ┌──────────────┐
            │  END point   │
            └──────┬───────┘
                   │
                   ▼
┌─────────────────────────────────────────────────────────────────────┐
│ Final State (Merged)                                                 │
│ {"name": "web_server_01",                                           │
│  "status": "Address object 'web_server_01' validated successfully"} │
└─────────────────────────────────────────────────────────────────────┘
```

**That's the whole flow!** Data entered the graph, flowed through the node which processed it, and came out updated.

### Key Concepts Illustrated

1. **State Flows Through**: The initial state enters at START and flows through each node
2. **Partial Updates**: The node returns ONLY the fields it changes (just "status")
3. **Automatic Merging**: LangGraph automatically merges the node's return value with the existing state
4. **Immutability**: The original state isn't modified; a new merged state is created

### What Makes This Powerful?

The function logic could be anything! We chose a simple validation, but in production this would be:
- ✅ An actual SCM API call to validate address object configuration
- ✅ Checking address object type and value format (IP/netmask, FQDN, IP range)
- ✅ Verifying folder/device group membership
- ✅ Validating against naming conventions
- ✅ Checking for duplicate names
- ✅ Any other validation task you need!

### Testing With Different Inputs

Let's verify our graph works with different address object names:

In [ ]:
# Test with multiple different address object names
print("Testing with different SCM address object names:")
print("=" * 60)

test_addresses = [
    "database-pool",
    "DMZ-network",
    "vpn-gateway-01"
]

for addr_name in test_addresses:
    result = app.invoke({"name": addr_name, "status": ""})
    print(f"\n✅ {addr_name}")
    print(f"   Status: {result['status']}")

print("\n" + "=" * 60)
print("💡 The same graph works for any address object name!")
print("   This is the power of parameterized workflows.")

---

## 5.1 Exercise: SCM Address Object Validation

Time for your very first exercise! This is quite similar to what we just did, but now it's your turn.

**Challenge**: Create an extended SCM address object validation workflow

**Requirements**:
- Create a state that tracks: `address_name`, `folder`, and `validation_status`
- Pass in an address name (like "web_server") and folder name (like "Texas")
- Build a node that validates the address object and updates status
- Output something like: "Address object 'web_server' in folder 'Texas' is valid and ready for creation"

**Detailed Hints**:

1. **State Definition** (Step 1):
   ```python
   class ExtendedAddressState(TypedDict):
       """Your docstring here"""
       address_name: str      # Name of the address object
       folder: str            # SCM folder name (e.g., 'Texas', 'Shared')
       validation_status: str # Validation result message
   ```

2. **Node Function** (Step 2):
   - Function signature: `def validate_address_config(state: ExtendedAddressState) -> dict:`
   - Read both `address_name` and `folder` from state
   - Create a status message using both values
   - Return ONLY `{"validation_status": "your message here"}`

3. **Graph Building** (Steps 3-5):
   - Use `StateGraph(ExtendedAddressState)`
   - Add node with name like `"validate_config"`
   - Use `set_entry_point()` and `set_finish_point()` with your node name

4. **Testing** (Steps 7-8):
   - Invoke with: `{"address_name": "web_server", "folder": "Texas", "validation_status": ""}`
   - Check that result includes all three fields

**Success Criteria**:
- ✅ State has exactly 3 fields (address_name, folder, validation_status)
- ✅ Node function has proper docstring
- ✅ Node returns only validation_status (partial update)
- ✅ Graph compiles without errors
- ✅ Visualization shows START → your_node → END
- ✅ Invoke returns all fields with validation_status populated

**Common Mistakes to Avoid**:
- ❌ Don't return the entire state - return only the field you're updating
- ❌ Don't forget the docstring - it's important for future AI agent integration
- ❌ Node name in add_node() must match set_entry_point() and set_finish_point()
- ❌ Don't forget to compile before invoking

**Steps to complete**:
1. Define ExtendedAddressState class with three fields
2. Create validate_address_config node function
3. Build graph with StateGraph
4. Add the node with add_node()
5. Connect START and END with set_entry_point() and set_finish_point()
6. Compile the graph
7. Visualize with draw_mermaid_png()
8. Invoke and test with sample data

Try it yourself in the cell below!

In [ ]:
# Your code here!
# Try building the extended address validation workflow yourself

# Solution will be provided in a separate notebook, but try it first!

# Step 1: Define ExtendedAddressState

# Step 2: Create validate_address_config function

# Step 3: Build graph

# Step 4: Add node

# Step 5: Connect START and END

# Step 6: Compile

# Step 7: Visualize

# Step 8: Invoke and test


---

### Exercise Solution

**Try the exercise yourself first!** Scroll down only if you're stuck or want to check your solution.

<details>
<summary><b>Click here to reveal the solution</b></summary>

Here's a complete working solution:

In [ ]:
# ========================================
# EXERCISE SOLUTION
# ========================================

# Step 1: Define ExtendedAddressState
class ExtendedAddressState(TypedDict):
    """Extended state for SCM address object validation workflow.
    
    Tracks address object name, folder location, and validation status.
    """
    address_name: str       # Name of the address object
    folder: str             # SCM folder (e.g., 'Texas', 'Shared')
    validation_status: str  # Validation result message

print("✅ Step 1: ExtendedAddressState defined with 3 fields")

# Step 2: Create validate_address_config function
def validate_address_config(state: ExtendedAddressState) -> dict:
    """Node: Validate SCM address object configuration.
    
    Validates that an address object is properly configured for
    creation in the specified SCM folder.
    
    Args:
        state: Current state with address_name and folder
        
    Returns:
        dict: Partial state update with validation_status
    """
    # Read address name and folder from state
    addr_name = state["address_name"]
    folder_name = state["folder"]
    
    # Create validation message
    # In production, this would check SCM API for name conflicts,
    # validate folder exists, check naming conventions, etc.
    status = f"Address object '{addr_name}' in folder '{folder_name}' is valid and ready for creation"
    
    # Return ONLY the field we're updating
    return {"validation_status": status}

print("✅ Step 2: validate_address_config function created")

# Step 3: Build graph
exercise_graph = StateGraph(ExtendedAddressState)
print("✅ Step 3: StateGraph initialized")

# Step 4: Add node
exercise_graph.add_node("validate_config", validate_address_config)
print("✅ Step 4: Node added to graph")

# Step 5: Connect START and END
exercise_graph.set_entry_point("validate_config")
exercise_graph.set_finish_point("validate_config")
print("✅ Step 5: Entry and finish points connected")

# Step 6: Compile
exercise_app = exercise_graph.compile()
print("✅ Step 6: Graph compiled")

# Step 7: Visualize
print("\n📊 Step 7: Graph Visualization:")
display(Image(exercise_app.get_graph().draw_mermaid_png()))

# Step 8: Invoke and test
print("\n🚀 Step 8: Testing the graph:")
print("=" * 60)

test_result = exercise_app.invoke({
    "address_name": "web_server",
    "folder": "Texas",
    "validation_status": ""
})

print("\nInput:")
print("  address_name: 'web_server'")
print("  folder: 'Texas'")
print("  validation_status: ''")

print("\nOutput:")
print(f"  address_name: '{test_result['address_name']}'")
print(f"  folder: '{test_result['folder']}'")
print(f"  validation_status: '{test_result['validation_status']}'")

print("\n" + "=" * 60)
print("🎉 Exercise Complete!")
print("=" * 60)

print("\n💡 Key Takeaways:")
print("   ✅ State defines the data structure (3 fields)")
print("   ✅ Node function processes and returns partial updates")
print("   ✅ Graph orchestrates the flow (START → node → END)")
print("   ✅ Compile creates executable application")
print("   ✅ Invoke runs with initial state values")

print("\n🎓 You've successfully built your second LangGraph workflow!")
print("   Now you understand the complete pattern!")

---

## 5.2 Troubleshooting Common Beginner Errors

As you build your first graphs, you might encounter some common issues. Here are solutions:

### Error 1: KeyError when accessing state

**Problem:**
```python
KeyError: 'name'
```

**Cause:** Trying to access a state field that doesn't exist or wasn't passed in invoke()

**Solution:**
```python
# ❌ BAD - state field doesn't exist
def my_node(state: AddressObjectState) -> dict:
    return {"unknown_field": "value"}

# ✅ GOOD - use only fields defined in TypedDict
def my_node(state: AddressObjectState) -> dict:
    return {"status": "value"}  # 'status' is in AddressObjectState
```

### Error 2: Node function returns wrong type

**Problem:**
```python
TypeError: 'NoneType' object is not subscriptable
```

**Cause:** Node function returns None instead of dict

**Solution:**
```python
# ❌ BAD - forgot to return
def my_node(state: AddressObjectState) -> dict:
    status = "validated"
    # Missing return!

# ✅ GOOD - always return a dict
def my_node(state: AddressObjectState) -> dict:
    status = "validated"
    return {"status": status}  # Must return dict
```

### Error 3: Node name mismatch

**Problem:**
```python
ValueError: Node 'validate' not found in graph
```

**Cause:** Node name in set_entry_point() doesn't match add_node()

**Solution:**
```python
# ❌ BAD - names don't match
graph.add_node("validate_node", validate_function)
graph.set_entry_point("validate")  # Wrong name!

# ✅ GOOD - names must match exactly
graph.add_node("validate", validate_function)
graph.set_entry_point("validate")  # Same name
```

### Error 4: Forgot to compile

**Problem:**
```python
AttributeError: 'StateGraph' object has no attribute 'invoke'
```

**Cause:** Trying to invoke the graph before compiling

**Solution:**
```python
# ❌ BAD - invoking uncompiled graph
graph = StateGraph(AddressObjectState)
graph.add_node("validate", my_function)
result = graph.invoke({"name": "test"})  # Error!

# ✅ GOOD - compile first
graph = StateGraph(AddressObjectState)
graph.add_node("validate", my_function)
app = graph.compile()  # Compile to create runnable
result = app.invoke({"name": "test"})  # Now works!
```

### Error 5: Missing state fields in invoke()

**Problem:**
```python
KeyError: 'status'
```

**Cause:** invoke() doesn't include all state fields

**Solution:**
```python
# ❌ BAD - missing 'status' field
result = app.invoke({"name": "web_server"})

# ✅ GOOD - include all TypedDict fields
result = app.invoke({"name": "web_server", "status": ""})
```

### Error 6: SCM-Specific Exceptions (Coming in Later Notebooks)

**When you start working with actual SCM API calls** (Notebooks 104+), you'll encounter these specific exceptions:

```python
from scm.exceptions import (
    InvalidObjectError,       # Configuration data format is wrong
    NameNotUniqueError,       # Object name already exists in SCM
    ObjectNotPresentError,    # Object not found (404)
    MissingQueryParameterError,  # Required parameter missing (e.g., folder)
    ReferenceNotZeroError,    # Can't delete - object is referenced elsewhere
)
```

**Common SCM Error Patterns:**

```python
# Example: Handling SCM exceptions in a node
def create_address_node(state: AddressState) -> dict:
    """Create address object with proper error handling."""
    try:
        # Attempt to create address object
        client.address.create({
            "name": state["name"],
            "ip_netmask": state["ip_address"],
            "folder": "Texas"
        })
        return {"status": "success", "message": "Address created"}
        
    except NameNotUniqueError:
        # Address name already exists - can't create duplicate
        return {"status": "error", "message": "Address name already exists"}
        
    except InvalidObjectError as e:
        # Configuration format is wrong (invalid IP, missing required field, etc.)
        return {"status": "error", "message": f"Invalid configuration: {e.message}"}
        
    except MissingQueryParameterError:
        # Forgot required parameter like folder
        return {"status": "error", "message": "Missing required parameter (folder)"}
        
    except Exception as e:
        # Catch-all for network issues, auth failures, etc.
        return {"status": "error", "message": f"Unexpected error: {str(e)}"}
```

**Why This Matters:**

- **Notebooks 104-107**: You'll implement actual SCM operations and need these exception handlers
- **Production Workflows**: Robust error handling prevents workflows from crashing
- **State-Based Error Tracking**: Errors are captured in state, allowing graphs to route to retry/recovery nodes

**Preview: Error Handling in Multi-Node Graphs**

In Notebook 106 (Conditional Routing), you'll use exception handling with conditional edges:

```python
def create_with_error_handling(state):
    try:
        client.address.create(state["config"])
        return {"status": "success"}
    except NameNotUniqueError:
        return {"status": "duplicate"}  # Routes to "update" node instead
    except InvalidObjectError:
        return {"status": "invalid"}    # Routes to "fix_config" node

# Conditional edge routes based on status
graph.add_conditional_edges(
    "create",
    lambda state: state["status"],
    {
        "success": "verify",
        "duplicate": "update_existing",
        "invalid": "fix_config"
    }
)
```

💡 **Pro Tip**: Always wrap SCM API calls in try-catch blocks and update state with error details. This allows your graph to make intelligent routing decisions!

### Debugging Tips

1. **Print state values** inside node functions:
   ```python
   def my_node(state: AddressObjectState) -> dict:
       print(f"DEBUG: Current state = {state}")
       # ... rest of function
   ```

2. **Check graph structure** with visualization:
   ```python
   display(Image(app.get_graph().draw_mermaid_png()))
   ```

3. **Test nodes independently** before adding to graph:
   ```python
   test_state = {"name": "test", "status": ""}
   result = validate_address_object(test_state)
   print(f"Node returns: {result}")
   ```

4. **Verify TypedDict fields** match everywhere:
   - State definition
   - invoke() input
   - Node function return values

5. **Test SCM operations with try-catch** (when you get to those notebooks):
   ```python
   try:
       result = client.address.create(config)
       print(f"✅ Success: {result.id}")
   except Exception as e:
       print(f"❌ Error: {type(e).__name__}: {e}")
   ```

💡 **Pro Tip**: When stuck, go back to basics - check each component individually!

---

## 6. Summary

Congratulations! You've just built your very first LangGraph application! 🎉

### What We Accomplished

1. **Defined State Structure**
   - Created `AddressObjectState` using TypedDict
   - Defined `name` and `status` fields to track address object information
   - Learned that state is just a class inheriting from TypedDict

2. **Created a Node Function**
   - Built `validate_address_object` as a standard Python function
   - Learned nodes take state as input and return partial state updates
   - Practiced using docstrings (important for AI agents later!)
   - Understood partial state updates vs full state replacement

3. **Built the Graph**
   - Created a StateGraph with our state schema
   - Added a node with a name and action
   - Connected START → node → END using set_entry_point and set_finish_point

4. **Compiled and Executed**
   - Compiled the graph into a runnable application
   - Visualized the graph structure using IPython mermaid diagrams
   - Invoked the graph with initial address object state
   - Saw how data flows through: input → node processing → output

### Key Takeaways

✅ **State** = Shared data structure (TypedDict class) tracking workflow information  
✅ **Node** = Python function that processes state and returns updates  
✅ **Graph** = StateGraph that manages the flow of execution  
✅ **Compile** = Creates executable application from graph definition  
✅ **Invoke** = Runs the graph with initial state values  

### The Pattern You'll Use Again and Again

```python
# 1. Define state
class AddressState(TypedDict):
    name: str
    status: str

# 2. Create node
def validate_address(state: AddressState) -> dict:
    """Validate address object configuration."""
    status = f"Address '{state['name']}' is valid"
    return {"status": status}

# 3. Build graph
graph = StateGraph(AddressState)
graph.add_node("validate", validate_address)
graph.set_entry_point("validate")
graph.set_finish_point("validate")

# 4. Compile and run
app = graph.compile()
result = app.invoke({"name": "web_server", "status": ""})
```

### Real-World Applications

The simple pattern you just learned applies to real SCM automation workflows:
- **Configuration validation**: Check address object, security rule, and policy settings
- **Compliance auditing**: Verify configurations meet security standards
- **Pre-deployment checks**: Validate configurations before pushing to production
- **Object creation workflows**: Validate and create SCM objects programmatically
- **Batch operations**: Process multiple configurations systematically

**Remember**: Don't worry if you didn't get 100% of everything - that's what the exercise is for! Practice solidifies understanding.

💡 **Pro Tip**: The pattern you just learned (state → node → graph → compile → invoke) is the foundation for ALL LangGraph applications, no matter how complex. Even sophisticated SCM automation workflows with AI agents follow this exact same pattern!

---

## 7. What's Next

Now that you've mastered the basics of single-node graphs, you're ready to:

**Notebook 104**: Multi-Node Graphs and Sequential Workflows
- Build graphs with multiple nodes (multi-step workflows)
- Learn how to chain nodes together sequentially
- Handle more complex state management across multiple processing steps
- Apply these patterns to real SCM workflows like address object creation

**Coming in later notebooks**:
- Add conditional routing (decision logic) with conditional edges
- Create looping workflows with retry logic
- Integrate tools and external APIs (actual SCM API calls)
- Eventually, build AI agents with LLMs that automate SCM operations!

Great work completing this foundation! See you in Notebook 104! 🚀
